# Create Cloud Provider Configuration
### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [ ]:
import configparser
import os
import http.client
import json
from pprint import pprint

platform_address = os.getenv('API_BASE_URL', "portal.dev.yugabyte.com")
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [ ]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

## GCP Provider
### Define GCP Provider

In this example, we will create a GCP provider. First, load in GCP service account credentials JSON file. Then, define a `Provider` object with the credentials and desired configuration.

In [ ]:
gcp_creds_filename = os.getenv('GCE_CREDENTIALS_FILE_PATH')
gcp_creds_file = open(gcp_creds_filename, 'r')
gcp_creds = json.loads(gcp_creds_file.read())

new_provider = {
    "code": "gcp",
    "config": {
        **gcp_creds,
        "YB_FIREWALL_TAGS": "cluster-server",
    },
    "destVpcId": "yugabyte-network",
    "name": "my-gcp-provider",
    "regions": [
        {
            "code": "us-central1",
            "name": "us-central1",
            "zones": [],
        },
    ],
    "sshPort": 54422,
    "airGapInstall": False,
}

### Create Provider

Make API call to provider endpoint to create new provider.

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("POST", url, json.dumps(new_provider), headers)
res = conn.getresponse()
print(json.dumps(json.load(res), indent=2))

## AWS Provider
### Define AWS Provider

In this example, we will create an AWS provider. You need to have set `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` environment variables.

In [ ]:
config = configparser.RawConfigParser()
config.read(os.path.expanduser('~/.aws/credentials'))
# print (config.sections())
new_provider = {
  "code": "aws",
  "name": "my-aws-provider",
  "config": {
    "AWS_ACCESS_KEY_ID": f"{config.get('default', 'aws_access_key_id')}",
    "AWS_SECRET_ACCESS_KEY": f"{config.get('default', 'aws_secret_access_key')}"
  },
  "regions": [
      {
          "code": "us-west-2",
          "name": "us-west-2",
          "zones": [],
      },
  ]
}
#pprint(new_provider)

### Create Provider

Make API call to provider endpoint to create new provider.

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("POST", url, json.dumps(new_provider), headers)
res = conn.getresponse()
pprint(json.load(res))

## Azure Provider
### Define Azure Provider

In this example, we will create a GCP provider. You need to have set `AZURE_CLIENT_ID`, `AZURE_CLIENT_SECRET`, `AZURE_TENANT_ID`, `AZURE_SUBSCRIPTION_ID`, and `AZURE_RG` environment variables.

In [ ]:

new_provider = {
  "code": "azu",
  "name": "my-azu-provider",
  "config": {
    "AZURE_CLIENT_ID": f"{os.getenv('AZURE_CLIENT_ID')}",
    "AZURE_CLIENT_SECRET": f"{os.getenv('AZURE_CLIENT_SECRET')}",
    "AZURE_TENANT_ID": f"{os.getenv('AZURE_TENANT_ID')}",
    "AZURE_SUBSCRIPTION_ID": f"{os.getenv('AZURE_SUBSCRIPTION_ID')}",
    "AZURE_RG": f"{os.getenv('AZURE_RG')}"
  },
  "regions": [
    {
      "code": "westus2",
      "vNetName": "yugabyte-vnet-us-west2",
      "zones": [
        {
          "name": "westus2-1",
          "subnet": "yugabyte-subnet-westus2"
        },
        {
          "name": "westus2-2",
          "subnet": "yugabyte-subnet-westus2"
        },
        {
          "name": "westus2-3",
          "subnet": "yugabyte-subnet-westus2"
        }
      ]
    },
  ]
}
#pprint(new_provider)

### Create Provider

Make API call to provider endpoint to create new provider.

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("POST", url, json.dumps(new_provider), headers)
res = conn.getresponse()
pprint(json.load(res))

## Onprem Provider
### Define Onprem Provider

In this example, we will create a Onprem provider. You will need to have a VM instance for db node and ssh config details for the db node instance.

### Create Provider

Make API call to providers UI endpoint.

In [ ]:
provider_ui = {
    "code": "onprem",
    "name": "onprem-provider-1",
    "config": {}
}
# pprint(provider_ui)

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers/ui"
conn.request("POST", url, json.dumps(provider_ui), headers)
res = conn.getresponse()
ui_response = json.load(res)
# pprint(ui_response)
provider_uuid = ui_response["uuid"]

Make API call to providers instance types endpoint.

In [ ]:

provider_instance_types = {
    "idKey": {
        "providerCode": "onprem",
        "instanceTypeCode": "c5.large"
    },
    "numCores": "2",
    "memSizeGB": "4",
    "instanceTypeDetails": {
        "volumeDetailsList": [
            {"volumeSizeGB": "16",
             "mountPath": "/data"
             }
        ]
    }
}
# pprint(provider_instance_types)

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/instance_types"
conn.request("POST", url, json.dumps(provider_instance_types), headers)
res = conn.getresponse()
# pprint(json.load(res))

Make API call to providers regions endpoint.

In [ ]:
provider_regions = {
    "code": "us-west-2",
    "hostVPCId": "",
    "name": "us-west-2",
    "latitude": "37",
    "longitude": "-121"
}
# pprint(provider_regions)

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/regions"
conn.request("POST", url, json.dumps(provider_regions), headers)
res = conn.getresponse()
regions_response = json.load(res)
# pprint(regions_response)
region_uuid = regions_response["uuid"]

Make API call to providers zones endpoint.

In [ ]:
provider_zones = {
    "availabilityZones": [
        {
            "code": "west1",
            "name": "west1"
        }
    ]
}
# pprint(provider_zones)

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/regions/{region_uuid}/zones"
conn.request("POST", url, json.dumps(provider_zones), headers)
res = conn.getresponse()
zones_response = json.load(res)
# pprint(zones_response)
zone_uuid = zones_response["west1"]["uuid"]

Make API call to providers access keys endpoint.

In [ ]:

HOME_DIR = os.path.expanduser("~")
key_file_path = os.path.join(HOME_DIR, '.yugabyte/yb-dev-aws-2.pem')
with open(key_file_path, "r") as f_in:
    key_file = f_in.read()
    provider_access_keys = {
        "keyCode": "onprem-provider-1-key",
        "regionUUID": region_uuid,
        "keyType": "PRIVATE",
        "keyContent": key_file,
        "sshUser": "centos",
        "sshPort": 22,
        "airGapInstall": False,
        "installNodeExporter": True,
        "nodeExporterUser": "prometheus",
        "nodeExporterPort": 9300,
        "skipProvisioning": False,
        "ntpServers": [],
        "setUpChrony": False
    }

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/access_keys"
conn.request("POST", url, json.dumps(provider_access_keys), headers)
res = conn.getresponse()
# pprint(json.load(res))

Make API call to providers add node endpoint.

In [ ]:
provider_add_node = {
    "nodes": [{ 
        "zone": "west1",
        "region": "us-west-2",
        "ip": "10.9.139.92",
        "instanceType": "c5.large",
        "sshUser": "centos",
        "sshPort": 22,
        "instanceName": "onprem-n1"
    }]
}

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/zones/{zone_uuid}/nodes"
conn.request("POST", url, json.dumps(provider_access_keys), headers)
res = conn.getresponse()
# pprint(json.load(res))